## TensorFlow_API

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential,metrics

In [2]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [3]:
batchsz = 128
(x, y), (x_val, y_val) = datasets.fashion_mnist.load_data()
db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(60000).batch(batchsz).repeat(10)

db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
db_val = db_val.map(preprocess).batch(batchsz, drop_remainder=True)

In [4]:
network = Sequential([   
    layers.Dense(256, activation=tf.nn.relu),    # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),    # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),     # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),     # [b, 64] => [b, 32]
    layers.Dense(10)                             # [b, 32] => [b, 10]
])

In [5]:
network.build(input_shape=(None, 28*28))
network.summary()

optimizer = optimizers.Adam(lr=0.001)

# [__1]创建metrics，用于求精确度
acc_meter = metrics.Accuracy()
# [1]创建metrics，用于求平均值
loss_meter = metrics.Mean()

for step, (x, y) in enumerate(db):
    with tf.GradientTape() as tape:
        x = tf.reshape(x, (-1, 28*28))
        out = network(x)
        y_onehot = tf.one_hot(y, depth=10)
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))
        
        # [2]更新loss_meter 中的数据，添加数据
        loss_meter.update_state(loss)
                
    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))
    
    if step % 100 == 0:
        # [3]获取loss_meter的值（loss的没100个的平均值），转换为numpy类型并打印输出
        print(step, 'loss:', loss_meter.result().numpy())
        # [4]清除所创建的loss_meter的buffer
        loss_meter.reset_states()
        
    # evaluate测试，每500步计算一次精度
    if step % 500 == 0:
        total, total_correct =0., 0.
        # [__2]需要先清空acc_meter
        acc_meter.reset_states()
        
        for step, (x, y) in enumerate(db_val):
            x = tf.reshape(x, [-1, 28*28])
            out = network(x)
            pred = tf.cast(tf.argmax(out, axis=1),tf.int32)
            correct = tf.equal(pred, y)
            total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
            total += x.shape[0]
            # [__3]更新acc_meter中的数据,添加数据
            acc_meter.update_state(y, pred)
        # [__4]获得acc_meter中的数据，并转化为numpy类型，最后打印输出    
        print(step,'Evaluate Acc: ', total_correct/total, acc_meter.result().numpy())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
0 loss: 2.3432314
77 Evaluate Acc:  0.14022435897435898 0.14022435
100 loss: 0.86405975
200 loss: 0.5323111
300 loss